## GPU Info & Mounting

In [ ]:
## GPU info

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Jul 29 07:28:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
## Mount Drive

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')
# root_path = 'drive/My Colab Notebooks/'  #change dir to your project folder

## Install & Imports

In [ ]:
import os
home = "/content"
os.chdir(home)
os.getcwd()

'/content'

In [ ]:
!pip install torch==1.6.0 torchvision==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


### Install MMF from Source

In [ ]:
!git clone --branch no_feats --config core.symlinks=true https://github.com/rizavelioglu/mmf.git

fatal: destination path 'mmf' already exists and is not an empty directory.


In [ ]:
import os
os.chdir(os.path.join(home, "mmf"))

In [ ]:
!pip install --editable .

Obtaining file:///content/mmf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Attempting uninstall: mmf
    Found existing installation: mmf 1.0.0rc12
    Can't uninstall 'mmf'. No files were found to uninstall.
  Running setup.py develop for mmf


## Convert to MMF format

In [ ]:
zip_file_path="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/data/hateful_meme_data.zip"

In [ ]:
!mmf_convert_hm --zip_file=$zip_file_path --password="" --bypass_checksum 1

2021-07-29 08:00:45.076813: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Data folder is /root/.cache/torch/mmf/data
Zip path is /content/drive/MyDrive/Colab_Notebooks/hateful_memes/data/hateful_meme_data.zip
Copying /content/drive/MyDrive/Colab_Notebooks/hateful_memes/data/hateful_meme_data.zip
Unzipping /content/drive/MyDrive/Colab_Notebooks/hateful_memes/data/hateful_meme_data.zip
Extracting the zip can take time. Sit back and relax.
Moving train.jsonl
Moving dev_seen.jsonl
Moving test_seen.jsonl
Moving dev_unseen.jsonl
Moving test_unseen.jsonl
Moving img


In [ ]:
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/ | wc -l

12141


## Feature Extraction

### VQA Mask-RCNN

In [ ]:
import os
os.chdir(home)
!git clone https://gitlab.com/vedanuj/vqa-maskrcnn-benchmark

In [ ]:
!pip install ninja yacs cython matplotlib

In [ ]:
os.chdir(os.path.join(home, "vqa-maskrcnn-benchmark"))
!rm -rf build
!python setup.py build develop

### Extract

#### Process Misogynistic Dataset

In [ ]:
# read the .jsonl file and get the img column
labeled_memo_samples = pd.read_json(os.path.join(home, "Misogynistic-MEME/train_miso.jsonl"), lines=True)['img']
# parse the img entries and get the image names
labeled_memo_samples = [i.split('/')[1] for i in list(labeled_memo_samples)]

img_dir = os.path.join(home, f"Misogynistic-MEME/img/")
for img in labeled_memo_samples:
    os.rename(f"{img_dir+img}", f"/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/{img}")

In [13]:
# Load labeled misoginistis
train = pd.read_json("/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/annotations/train.jsonl", lines=True)
miso = pd.read_json(os.path.join(home, "Misogynistic-MEME/train_miso.jsonl"), lines=True)
# Add miso data to 'train.jsonl'
train = pd.concat([train, miso], axis=0)

In [14]:
# Shuffle data
train = train.sample(frac=1).reset_index(drop=True)
# Write new jsonl file
train_json = train.to_json(orient='records', lines=True)

with open(os.path.join(home, "train_v11.jsonl"), "w", encoding='utf-8') as f:
    f.write(train_json)

In [ ]:
os.chdir(os.path.join(home, "mmf/tools/scripts/features/"))
out_folder = os.path.join(home, "features/")

!python extract_features_vmb.py --config_file "https://dl.fbaipublicfiles.com/pythia/detectron_model/detectron_model_x152.yaml" \
                                --model_name "X-152" \
                                --output_folder $out_folder \
                                --image_dir "/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/" \
                                --num_features 100 \

In [ ]:
os.chdir(home)
# !zip -r "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/features.zip" features

## Fine-tuning w/ VisualBERT

In [ ]:
# os.chdir("/content/drive/MyDrive/Colab Notebooks/hateful_memes/")
# os.getcwd()
# # !unzip "features.zip"

'/content/drive/MyDrive/Colab Notebooks/hateful_memes/features'

In [ ]:
os.chdir(home)
os.getcwd()

'/content'

In [ ]:
log_dir ="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/logs/baseline/visual_bert"
save_dir="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/submissions/baseline/visual_bert/submission_1/"
feats_dir = "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/features/"
train_dir = "hateful_memes/defaults/annotations/train.jsonl"


### Run Model with Default Config

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir $log_dir

In [ ]:

# # Define where train.jsonl is
# train_dir = os.path.join(home, "train_v9.jsonl")
!mmf_run \
        config="projects/visual_bert/configs/hateful_memes/from_coco.yaml" \
        model="visual_bert" \
        dataset=hateful_memes \
        run_type=train_val \
        checkpoint.max_to_keep=1 \
        checkpoint.resume_zoo=visual_bert.pretrained.cc.full \
        training.tensorboard=True \
        training.checkpoint_interval=100 \
        training.evaluation_interval=100 \
        training.max_updates=3000 \
        training.log_interval=100 \
        dataset_config.hateful_memes.max_features=100 \
        dataset_config.hateful_memes.annotations.train[0]=$train_dir \
        dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_unseen.jsonl \
        dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_unseen.jsonl \
        dataset_config.hateful_memes.features.train[0]=$feats_dir \
        dataset_config.hateful_memes.features.val[0]=$feats_dir \
        dataset_config.hateful_memes.features.test[0]=$feats_dir \
        training.lr_ratio=0.3 \
        training.use_warmup=True \
        training.batch_size=32 \
        optimizer.params.lr=5.0e-05 \
        env.save_dir=$save_dir \
        env.tensorboard_logdir=$log_dir \

### Run VisualBERT Model with Hyper-Parameter Sweep x27

In [ ]:
%tensorboard --logdir "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/hyperparameter_sweep/logs/"

In [ ]:
hyper_params_path="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/hyperparameter_sweep"

In [ ]:
os.chdir(hyper_params_path)
# Give rights to bash script to be executable
!chmod +x sweep.sh
# Define where image features are
feats_dir = "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/features/"
# Define where train.jsonl is
train_dir = "hateful_memes/defaults/annotations/train.jsonl"

# Start hyper-parameter search
!python sweep.py --home $home --feats_dir $feats_dir --train $train_dir

### Run Single Model with Modified Hyper-parameter Configs

#### Early Stopping: True
#### FP16 Precision: True
#### Max Updates: 1000
#### LR Ratio: 0.1
#### Training Warmup: False

In [ ]:
os.chdir(home)
os.getcwd()

'/content'

In [ ]:
%tensorboard --logdir "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/logs/"

In [ ]:
tensorboard_log_dir ="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/logs/"
save_dir="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/submissions/"
feats_dir = "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/features/"
train_dir = "hateful_memes/defaults/annotations/train.jsonl"

In [ ]:
!mmf_run \
        config="projects/visual_bert/configs/hateful_memes/from_coco.yaml" \
        model="visual_bert" \
        dataset=hateful_memes \
        run_type=train_val \
        checkpoint.max_to_keep=1 \
        checkpoint.resume_zoo=visual_bert.pretrained.cc.full \
        training.tensorboard=True \
        training.checkpoint_interval=50 \
        training.evaluation_interval=50 \
        training.max_updates=3000 \
        training.log_interval=100 \
        training.lr_ratio=0.1 \
        training.use_warmup=False \
        training.batch_size=40 \
        training.evaluate_metrics=True \
        training.early_stop.enabled=True \
        training.early_stop.criteria='hateful_memes/roc_auc' \
        training.early_stop.minimize=False \
        training.fp16=True \
        dataset_config.hateful_memes.max_features=100 \
        dataset_config.hateful_memes.annotations.train[0]=$train_dir \
        dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_unseen.jsonl \
        dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_unseen.jsonl \
        dataset_config.hateful_memes.features.train[0]=$feats_dir \
        dataset_config.hateful_memes.features.val[0]=$feats_dir \
        dataset_config.hateful_memes.features.test[0]=$feats_dir \
        env.save_dir=$save_dir \
        env.tensorboard_logdir=$tensorboard_log_dir \

## Fine-Tuning via ViLBERT

In [ ]:
import os
os.chdir(home)
os.getcwd()

'/content'

In [ ]:
log_dir ="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/logs/baseline/vilbert/"
# save_dir="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/submissions/baseline/vilbert/"
save_dir="/content/submissions/baseline/vilbert/"
feats_dir = "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/features/"
train_dir = "hateful_memes/defaults/annotations/train.jsonl"

In [ ]:
!kill 235

In [ ]:
%tensorboard --logdir "/content/drive/MyDrive/Colab_Notebooks/hateful_memes/logs/baseline/"

In [ ]:
!mmf_run \
        config="projects/vilbert/configs/hateful_memes/from_cc.yaml" \
        model="vilbert" \
        dataset=hateful_memes \
        run_type=train_val \
        checkpoint.max_to_keep=1 \
        checkpoint.resume_zoo=vilbert.finetuned.hateful_memes.from_cc_original \
        training.tensorboard=True \
        training.checkpoint_interval=50 \
        training.evaluation_interval=50 \
        training.max_updates=5000 \
        training.log_interval=100 \
        dataset_config.hateful_memes.max_features=100 \
        dataset_config.hateful_memes.annotations.train[0]=$train_dir \
        dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_unseen.jsonl \
        dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_unseen.jsonl \
        dataset_config.hateful_memes.features.train[0]=$feats_dir \
        dataset_config.hateful_memes.features.val[0]=$feats_dir \
        dataset_config.hateful_memes.features.test[0]=$feats_dir \
        training.lr_ratio=0.3 \
        training.use_warmup=True \
        training.batch_size=40 \
        training.evaluate_metrics=True \
        optimizer.params.lr=5.0e-05 \
        env.save_dir=$save_dir \
        env.tensorboard_logdir=$log_dir \

## Hyper-Parameter Sweep with Best Config of VisualBERT + Dataset Expansion

In [ ]:
# !unzip '/content/drive/MyDrive/Colab_Notebooks/hateful_memes/misogynistic_meme_features/features_miso.zip' -d '/content/drive/MyDrive/Colab_Notebooks/hateful_memes/misogynistic_meme_features/features'

In [ ]:
# import sys
# sys.path.append('/content/drive/MyDrive/Colab_Notebooks/hateful_memes')
# from hparams_sweep import run_sweep
# from subprocess import call

In [ ]:
train_dir="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/misogynistic_meme_features/train_miso.jsonl"
feats_dir="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/misogynistic_meme_features/features/content/features"
log_dir="/content/drive/MyDrive/Colab_Notebooks/hateful_memes/logs/best/visual_bert"
save_dir="/content/submissions/best/visual_bert"

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir $log_dir

In [ ]:
hparams = {
        "batch_size": [30, 40, 50],
        "learning_rate": [5e-3, 5e-5, 5e-6],
        "lr_ratio": [0.1, 0.3, 0.5]
}

In [ ]:
!OC_DISABLE_DOT_ACCESS_WARNING=1

for batch_size in hparams['batch_size']:
        for lr in hparams['learning_rate']:
            for ratio in hparams['lr_ratio']:

              log_name = f'{batch_size}_{lr}_{ratio}'
              log_path = f'{log_dir}/{log_name}'
              save_path = f'{save_dir}/{log_name}'

              print('==================================================')
              print(f'Current: {log_name}')
              print('==================================================\n')

              !mmf_run \
                config="projects/visual_bert/configs/hateful_memes/from_coco.yaml" \
                model="visual_bert" \
                dataset=hateful_memes \
                run_type=train_val \
                checkpoint.max_to_keep=1 \
                checkpoint.resume_zoo=visual_bert.pretrained.cc.full \
                training.tensorboard=True \
                training.checkpoint_interval=100 \
                training.evaluation_interval=100 \
                training.max_updates=3000 \
                training.log_interval=100 \
                training.early_stop.enabled=True \
                training.early_stop.criteria='hateful_memes/roc_auc' \
                training.early_stop.minimize=False \
                dataset_config.hateful_memes.max_features=100 \
                dataset_config.hateful_memes.annotations.train[0]=$train_dir \
                dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_unseen.jsonl \
                dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_unseen.jsonl \
                dataset_config.hateful_memes.features.train[0]=$feats_dir \
                dataset_config.hateful_memes.features.val[0]=$feats_dir \
                dataset_config.hateful_memes.features.test[0]=$feats_dir \
                training.lr_ratio=$ratio \
                training.use_warmup=True \
                training.batch_size=$batch_size \
                optimizer.params.lr=$lr \
                env.save_dir=$save_path\
                env.tensorboard_logdir=$log_path 
              
              print('==================================================')
              print('Deleting model files for space saving')
              !rm -rf $save_path
              print('==================================================\n')
              
              

Streaming output truncated to the last 5000 lines.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)

WARNING 2021-07-29T13:55:19 | py.warnings: /usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)

2021-07-29T13:55:38 | mmf.trainers.callbacks.logistics: progress: 3000/3000, train/hateful_memes/cross_entropy: 0.0963, train/hateful_memes/cross_entropy/avg: 0.2355, train/total_loss: 0.0963, train/total_loss/avg: 0.2355, max mem: 8674.0, experiment: run, epoch: 10, num_updates: 3000, iterations: 3000, max_updates: 3000, lr: 0., ups: 1.06, time: 01m 34s 137ms, time_since_start: 01h 06m 44s 124ms, eta: 0ms
2021-07-29T13:55:38 | mmf.trainers.core.training_loop: Evaluation time. Running on full validation set...
WARNING 2021-07-29T13:55:51 | py.warnings: /usr/local/lib/python3.7/dist-packages/omegaconf/dictconfig.py:252: UserW